# Smartphones and Watches advisor
In this serie of notebooks we will:
- Load text from a Youtube video playlist
- Split loaded text into appropriate chunks
- **Create a vector store to load chunk embeddings**
- Create a Question Answering chatbot:

    - with different prompts techniques
    - with different similarity measures
    - with or without chat memory 

!['Data Connection'](../../images/les_numeriques_youtube/data_connection.png)


# Create a vector store to load chunk embeddings

We will do the following steps:
- Embed the chunks
- Store them in a Vector Store
- Apply two retrieval techniques:
    - Semantic similarity
    - Maximum marginal relevance

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
# Imports 
# Env var
import sys
from dotenv import load_dotenv, find_dotenv

# Langchain
from langchain.vectorstores import Chroma, MyScale, MyScaleSettings
from langchain.embeddings.openai import OpenAIEmbeddings

# MyScale
from clickhouse_connect.driver.exceptions import DatabaseError

# Save loader
from utils import load_docs_from_jsonl, save_docs_to_jsonl, count_lines_in_myscale

In [3]:
PATH_NAME_SPLITTER = './splitted_docs.jsonl'

In [4]:
# Env variable
sys.path.append('../..')
load_dotenv(find_dotenv())

True

In [5]:
docs = load_docs_from_jsonl(PATH_NAME_SPLITTER)

Let's look at the first chunk

In [6]:
docs[0]

Document(page_content="Bonjour à tous et bienvenue sur Les Numériques. Alors, comme chaque année, le mois de septembre, c'est réservé aux iPhones. Et 2022, évidemment, ne déroge pas la règle. Cette année, on aura vu débarquer la série 14 avec les iPhones 14 et les iPhones 14 Pro. Alors, dans cette vidéo aujourd'hui, on va plutôt s'intéresser aux versions Pro. On reviendra évidemment sur les 14 classiques lors de la sortie de la variante Plus de cet octobre. Mais bref, trêve de bavardage.", metadata={'source': 'docs/youtube/[TEST] iPhone 14 Pro et Pro Max ： la maturité avant les changements ？.m4a', 'chunk': 0})

In [7]:
# Embedding model
embedding = OpenAIEmbeddings()

### Chroma

In [8]:
# Vector store 
persist_directory = 'docs/chroma/'
!rm -rf ./docs/chroma  # remove old database files if any
vectordb = Chroma.from_documents(
    documents=docs,
    embedding=embedding,
    persist_directory=persist_directory
)

print(vectordb._collection.count())

815


### Question retrieval 

In [9]:
question = """Quelles sont les caractéristiques de l'iPhone 14 ?"""

## Semantic similarity search
[LangChain doc](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/similarity) 

This object selects examples based on **similarity** to the inputs. 

It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs.

In [13]:
ss_res = vectordb.similarity_search(
    question,
    k=5
)

ss_res

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a", 'chunk': 0}),
 Document(page_content="Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sauf que l'on retrouve la puce Apple A15 Bionic que l'on retrouvait dans les iPhone 13 Pro et 13 Pro Max en 2021. Du côté de l'iPhone 14, on garde le même gabarit qu'avec l'iPhone 13 et du côté de l'iPhone 14+, on prend le même gabarit qu'un iPhone 14 Pro Max.

## Maximum marginal relevance (MMR)
[LangChain doc](https://python.langchain.com/docs/modules/model_io/prompts/example_selectors/mmr) 

The **Maximum marginal relevance** selects examples based on a combination of which examples are most similar to the inputs, while also optimizing for diversity. 

It does this by finding the examples with the embeddings that have the greatest cosine similarity with the inputs, and then iteratively adding them while penalizing them for closeness to already selected examples.

![MMR.png](../../images/les_numeriques_youtube/MMR.png)

In [14]:
mmr_res = vectordb.max_marginal_relevance_search(
    question,
    k=5,
    fetch_k=20
)

mmr_res

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a", 'chunk': 0}),
 Document(page_content="On a donc un écran de 6,7 pouces, mais pour le reste, c'est exactement la même fiche technique qu'un iPhone 14. La batterie a été améliorée, mais ça c'est grâce à la nouvelle puce. En photo, très peu de changements, si ce n'est qu'on récupère les capteurs qui étaient présents sur les iPhone 13 Pro pour le principal et pour l'ultra grand-angle. Le module frontal a été entièrement revu.", metadata={'source': "docs/youtub

In [16]:
print("\n MMR Extra chunk \n")
print("\n\n".join([str(chunk) for chunk in mmr_res if chunk not in ss_res]))

print("\n SS Extra chunk \n")
print("\n\n".join([str(chunk) for chunk in ss_res if chunk not in mmr_res]))


 MMR Extra chunk 

page_content="Alors, premier point, le design. Vous aurez remarqué que les iPhone 13 ressemblent beaucoup aux iPhone 12 et au jeu des sept différences, il n'y a pas grand-chose à relever. Alors, je ne sais pas pour toi, mais moi, à part l'encoche et un point un petit peu plus élevé, j'ai peu de choses à dire. Donc l'encoche est un petit peu plus réduite sur les iPhone 13, tout modèle confondu. Et puis à part ça, c'est tout. Sur les iPhone 13 Pro et Pro Max, il y a quand même d'autres choses à relever." metadata={'source': 'docs/youtube/Test iPhone 13 ： notre verdict sur toute la gamme.m4a', 'chunk': 0}

page_content="D'après certaines rumeurs, les iPhone 13 posséderaient une bobine de cuivre plus large que sur les iPhone 12, ce qui permettrait d'offrir une charge rapide sans fil plus rapide que l'an passé, notamment grâce à la technologie MagSafe qui a été introduite avec les iPhone 12 l'année passée. Pour terminer sur le gros des nouveautés des iPhone 13, comment n

In [17]:
# Persist the vector database
vectordb.persist()

# MyScale

**MyScale** est une vector database hébergée sur le Cloud, ayant un free-tier.
Nous nous sommes créés un compte pour ce tutoriel, et un cluster nommé `les_numeriques_youtube`.
Depuis l'interface web de **MyScale**, nous aurons pu récupérer les paramètres de connection du cluster.

Ceux-ci sont à rentrer dans le fichier `.env` avec le préfixe `MYSCALE_`:
- `MYSCALE_HOST`
- `MYSCALE_PORT`
- `MYSCALE_USERNAME`
- `MYSCALE_PASSWORD`

L'objet `MyScaleSettings` pourra récupérer ces paramètres en lui fournissant le lien du fichier `.env`.


In [18]:
settings = MyScaleSettings(_env_file=find_dotenv())

myscale = None
try:
    # Let's check that we do not already have a database or that it's not empty
    myscale = MyScale(embedding, settings)
    
    count = count_lines_in_myscale(myscale)
    print(f"Nombre de lignes dans la table: {count}")
    if count == 0:
        raise ValueError("Il devrait y avoir des lignes dans la base de donnée, or il n'y en a pas.")
except (DatabaseError, ValueError):
    # No table exist
    print("Création de la table")
    
    myscale = MyScale.from_documents(
        documents=docs,
        embedding=embedding,
        config=settings
    )
finally:
    count = count_lines_in_myscale(myscale)
    print(f"Nombre de lignes finales dans la table: {count}")

Nombre de lignes dans la table: 815
Nombre de lignes finales dans la table: 815


Une table nommée `langchain` est ainsi créée dans le Cluster. Voilà un aperçu dans le **SQL Workspace**

![myscale_sql_workspace.png](../../images/les_numeriques_youtube/myscale_sql_workspace.png)

In [19]:
myscale.similarity_search(question, k=5)

[Document(page_content="Comme prévu lors de sa canote de rentrée, Apple a annoncé non pas un, non pas deux, non pas trois, mais bien quatre iPhones. Pas d'iPhone mini au programme, mais ça c'était prévu. A la place on se retrouve avec un iPhone 14, un iPhone 14+, un iPhone 14 Pro et un iPhone 14 Pro Max et je vous propose qu'on décortique tout ça maintenant. On va commencer avec les deux entrées de gamme qui sont les iPhone 14 et les iPhone 14+.", metadata={'chunk': 0, 'source': "docs/youtube/iPhone 14 ： la gamme Pro gâtée, l'autre oubliée.m4a"}),
 Document(page_content="Alors esthétiquement, il n'y a rien de nouveau par rapport aux iPhone 13 mini et aux iPhone 13 de l'année dernière. A l'intérieur, c'est quasiment la même chose, sauf que l'on retrouve la puce Apple A15 Bionic que l'on retrouvait dans les iPhone 13 Pro et 13 Pro Max en 2021. Du côté de l'iPhone 14, on garde le même gabarit qu'avec l'iPhone 13 et du côté de l'iPhone 14+, on prend le même gabarit qu'un iPhone 14 Pro Max.